In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import glob

#表示行数の増加
pd.set_option('display.max_rows', 1000)

today = datetime.datetime.now()

y = today.year
m = today.month - 1

ym = str(y) + str('{0:02d}'.format(m))

#暫定でのroot dir
root = "/Users/miyamoto/Documents/Root/raw"

In [2]:
#共通関数群

def file_ditect_direct(key_dir):#指定ディレクトリに存在するファイルパスの一覧取得
    path = root + "/" + key_dir
    
    files = []

    # パスで指定したファイルの一覧をリスト形式で取得
    files = glob.glob(path + '/*.csv')

    #print(files)

    for file in files:
        print(file)
    
    return files

def effectiveStock_load(df):
    all_stock = df.query("実在庫数 >= 1 & 受注可能在庫数 >= 1")
    
    order_in_3W = all_stock.query("受注数 >= 1")
    
    order_out_of_3W = all_stock.query("受注数 == 0 & 入庫数 >= 1")
    
    return all_stock, order_in_3W, order_out_of_3W

def stockReport_load(df):
    df = df.query("受注可能在庫数 >= 1 & 販売停止ステータス == '販売中' ")
    
    remove_unnecessary = df[~(df["商品名"].str.contains("テスト") | df["商品名"].str.contains("ダミー"))]
    
    remove_unnecessary["販売単価"] = \
    remove_unnecessary["販売単価"].where(remove_unnecessary["販売単価"] != '< NULL >',remove_unnecessary["定価"]).astype("float64")
    
    remove_unnecessary["受注可能在庫金額"] = remove_unnecessary["販売単価"] * remove_unnecessary["受注可能在庫数"]
    remove_unnecessary["属性"] = "予約中在庫"
    
    return remove_unnecessary

def import_csv(df, file):
    if "稼働在庫" in file:
        all_stock, order_in_3W, order_out_of_3W = effectiveStock_load(df)
        return all_stock, order_in_3W, order_out_of_3W
    elif "在庫明細" in file:
        df = stockReport_load(df)
        return df


In [3]:
def import_effectiveStock(file):
    effectiveStock = pd.read_csv(file)
    all_stock, order_in_3W, order_out_of_3W = import_csv(effectiveStock, file)
    
    return all_stock, order_in_3W, order_out_of_3W

def import_stockReport(file):
    reportStock = pd.read_csv(file)
    reportStock = import_csv(reportStock, file)
    
    return reportStock

def agg_func(df):
    df_agg = \
    df.groupby(["属性", "事業部ID", "事業部名", "ブランドコード", "ブランド名"])\
    .agg({'実在庫数':'sum', '実在庫金額':'sum', '受注可能在庫数':'sum', '受注可能在庫金額':'sum', "商品コード":"nunique", "SKUコード":"nunique"})
    
    return df_agg

def import_Arpege_allstock(file):
    df = pd.read_csv(file)
    
    df = df[~(df["商品名"].str.contains("テスト") | df["商品名"].str.contains("ダミー"))]
    df["属性"] = "Arpege総在庫"
    df["販売単価"] = df["販売単価"].where(df["販売単価"] != '< NULL >',df["定価"]).astype("float64")
    
    df["受注可能在庫金額"] = df["販売単価"] * df["受注可能在庫数"]
    
    df = df.query("受注可能在庫数 > 0")
    
    df_agg = \
    df.groupby(["属性", "事業部名", "ブランドコード"]).agg({'受注可能在庫金額':'sum'})
    
    return df, df_agg

def Arpege_in3W(ap_stockRaw, ap_order):
    ap_order = ap_order.groupby(["SKUコード"]).agg({'受注金額':'sum'})
    ap_stock_in3W = pd.merge(ap_stockRaw, ap_order, on=["SKUコード"])
    
    ap_stock_in3W["属性"] = "Arpege3W受注あり"
    
    ap_stock_in3W = \
    ap_stock_in3W.groupby(["属性", "事業部名", "ブランドコード"]).agg({'受注可能在庫金額':'sum', "SKUコード":"nunique"})
    
    return ap_stock_in3W

def Arpege_in3W_receive(ap_stockRaw, ap_order, ap_receive):
    ap_order = ap_order.groupby(["ブランドコード", "商品コード", "SKUコード"]).agg({'受注金額':'sum'})
    ap_receive = ap_receive.groupby(["ブランドコード", "商品コード", "SKUコード"]).agg({'入庫実績':'sum'})
    
    ap_stock_in3W_receive = pd.merge(ap_stockRaw, ap_receive, on=["ブランドコード", "商品コード", "SKUコード"])
    ap_stock_in3W_receive = pd.merge(ap_stock_in3W_receive, ap_order, on=["ブランドコード", "商品コード", "SKUコード"], how = "left")
    ap_stock_in3W_receive = ap_stock_in3W_receive[ap_stock_in3W_receive['受注金額'].isnull()]
    
    ap_stock_in3W_receive["属性"] = "Arpege3W以内入庫"
    
    ap_stock_in3W_receive = \
    ap_stock_in3W_receive.groupby(["属性", "事業部名", "ブランドコード"])\
    .agg({'受注可能在庫金額':'sum', "SKUコード":"nunique"})
    
    return ap_stock_in3W_receive

In [4]:
files = file_ditect_direct("effectiveStock")
ap_files = file_ditect_direct("effectiveStock/Arpege")

/Users/miyamoto/Documents/Root/raw/effectiveStock/在庫明細履歴_20210201094921.csv
/Users/miyamoto/Documents/Root/raw/effectiveStock/★稼働在庫_20210201094722.csv
/Users/miyamoto/Documents/Root/raw/effectiveStock/Arpege/在庫明細_20210201095133.csv
/Users/miyamoto/Documents/Root/raw/effectiveStock/Arpege/入庫実績_20210201095109.csv
/Users/miyamoto/Documents/Root/raw/effectiveStock/Arpege/受注_20210201095244.csv


In [5]:
for file in files:
    if "稼働" in file:
        all_stock, order_in_3W, order_out_of_3W = import_effectiveStock(file)
    elif "在庫明細" in file:
        stockReport = import_stockReport(file)

all_stock["属性"] = "総在庫"
order_in_3W["属性"] = "3W受注あり"
order_out_of_3W["属性"] = "3W以内入庫"

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykern

In [6]:
for file in ap_files:
    if "在庫明細" in file:
        ap_stockRaw, ap_allStock = import_Arpege_allstock(file)
        ap_stockRaw.rename(columns={"在庫数（良品）":"在庫数"}, inplace = True)
    elif "受注" in file:
        ap_order = pd.read_csv(file)
        ap_order = ap_order.query("受注金額 != 0 & ショップ名 == 'Arpege story' ")
    elif "入庫" in file:
        ap_receive = pd.read_csv(file)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
ap_allStock

受注可能在庫金額
属性        事業部名            ブランドコード             
Arpege総在庫 Arpege story事業部 DC000    111715500.0
                          DC010     82945500.0
                          DC020     51424700.0
                          DC050     75128400.0
                          DC060     15964200.0
                          DC070     27833400.0
                          DCNOV            0.0

In [8]:
ap_stock_in3W = Arpege_in3W(ap_stockRaw, ap_order)
ap_stock_in3W.rename(columns = {"SKUコード":"SKU数"}, inplace = True)

In [9]:
ap_stock_in3W

受注可能在庫金額  SKU数
属性           事業部名            ブランドコード                  
Arpege3W受注あり Arpege story事業部 DC000    80708000.0   436
                             DC010    48298800.0   349
                             DC020    30725900.0   217
                             DC050    36730300.0   249
                             DC060     2428600.0    42
                             DC070     8370000.0   118

In [10]:
ap_stock_in3W_receive = Arpege_in3W_receive(ap_stockRaw, ap_order, ap_receive)
ap_stock_in3W_receive.rename(columns = {"SKUコード":"SKU数"}, inplace = True)

In [11]:
ap_stock_in3W_receive

受注可能在庫金額  SKU数
属性           事業部名            ブランドコード                  
Arpege3W以内入庫 Arpege story事業部 DC000     7576200.0    60
                             DC010     8306000.0    48
                             DC020     3460200.0    27
                             DC050    17569300.0    81
                             DC060     2005900.0    79
                             DC070     3875000.0    42
                             DCNOV           0.0     6

In [12]:
ap_order_preserve = ap_order.query("販売ステータス == '予約' ")
ap_order_preserve = ap_order_preserve.groupby(["ブランドコード", "商品コード", "SKUコード"]).agg({"受注金額":"sum"})
ap_stock_nothing = ap_stockRaw.query("受注可能在庫数 > 0 & 在庫数 == 0 & 販売停止ステータス == '販売中'")

ap_stock_nothing = ap_stock_nothing[~(ap_stock_nothing["商品名"].str.contains("テスト") | ap_stock_nothing["商品名"].str.contains("ダミー"))]
ap_stock_nothing["属性"] = "Arpege予約中在庫"
ap_stock_nothing["販売単価"] = ap_stock_nothing["販売単価"].where(ap_stock_nothing["販売単価"] != '< NULL >',ap_stock_nothing["定価"]).astype("float64")
ap_stock_nothing["受注可能在庫金額"] = ap_stock_nothing["販売単価"] * ap_stock_nothing["受注可能在庫数"]

ap_stock_nothing = pd.merge(ap_stock_nothing, ap_order_preserve, on=["ブランドコード", "商品コード", "SKUコード"])

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [13]:
ap_stock_nothing_agg = ap_stock_nothing.groupby(["属性", "事業部名", "ブランドコード"]).agg({'受注可能在庫金額':'sum'})

In [14]:
ap_stock_nothing_agg

受注可能在庫金額
属性          事業部名            ブランドコード            
Arpege予約中在庫 Arpege story事業部 DC000    21261400.0
                            DC010    19202000.0
                            DC020     7320000.0
                            DC050    10199000.0
                            DC060     1170000.0
                            DC070     2048000.0

In [15]:
all_agg = agg_func(all_stock)
order_in_3W_agg =agg_func(order_in_3W)
order_out_of_3W_agg = agg_func(order_out_of_3W)

In [16]:
df_all_effective = pd.concat([all_agg, order_in_3W_agg, order_out_of_3W_agg], axis = 0)
df_all_effective.rename(columns = {"商品コード":"商品数", "SKUコード":"SKU数"}, inplace = True)

In [17]:
df_all_effective

実在庫数  \
属性     事業部ID 事業部名                     ブランドコード ブランド名                                          
総在庫    AA002 株式会社AMS　TEST事業部          AMS01   AMS                                     1193   
                                      AMS02   PRAMS                                    320   
       AUR02 AURALEE事業部               AU010   AURALEE                                 1789   
       BRS02 Brown&Street事業部          BR010   Brown&Street                            1183   
       BTY02 BUTSU/YOKU事業部            BT002   TOKYO 2020 OFFICIAL LICENSED PRODUCT    1595   
                                      BT005   Apothecary Milano                         59   
                                      BT007   EASTFAREAST                              167   
                                      BT008   CRAMSHELL                                 21   
                                      BT009   Yeti                                     113   
                                      BT010   Austin Austin                              3   
                                      BT012   The Inoue Brothers…                      146   
                                      BT014   MONOEARTH                                 91   
       C0008 STRIPE CLUB事業部           CC010   earth music&ecology                    75328   
                                      CC020   E hyphen world gallery                 12057   
                                      CC023   E hyphen world gallery BonBon              1   
                                      CC030   SEVENDAYS=SUNDAY                          35   
                                      CC060   Green Parks                            34801   
                                      CC070   YECCA VECCA                             9768   
                                      CC081   Maison de FLEUR                        12150   
                                      CC08C   Maison de FLEUE Petite Robe             4250   
                                      CC08F   haluhiroine                             1338   
                                      CC095   ブランド共通                                     8   
                                      CC0AA   emutto／スナックエミュット                         375   
                                      CC0CA   SLURR                                    105   
                                      CC0F0   Samansa Mos2                             154   
                                      CC0F1   Te chichi                                343   
                                      CC0F2   Lugnoncure                               323   
                                      CC0F3   ehka sopo                                 90   
                                      CC0F5   Samansa Mos2 Lagom                        31   
                                      CC0F6   Samansa Mos2 boyish                      647   
                                      CC0F7   Melan Cleuge Women's                     124   
                                      CC0H0   AMERICAN HOLIC                         45615   
                                      CC0K0   koe                                    20267   
                                      CC0M1   NiCORON                                 3618   
                                      CC0MJ   Areeam                                  7151   
                                      CC0N0   STRIPE CLUB pop up store                  20   
                                      CC0N2   ストクラセレクト                                4506   
                                      CC0N7   rem closet                               916   
                                      CC0N9   STRIPE CLUB KIDS by smarby             20049   
                                      CC0PA   CRAFT STANDARD BOUTIQUE                14213   
                                      CC0QA   ノモナキス                                   1437   
                                      CC0RA 

In [18]:
df_all_effective.to_csv('稼働在庫.csv', encoding='utf_8_sig')

In [19]:
stockReport.rename(columns={"在庫数（良品）":"在庫数"}, inplace = True)

stockReport = stockReport.query("在庫数 == 0")

stockReport

,履歴日付,倉庫名,事業部名,ブランドコード,ブランド名,大カテゴリ名,中カテゴリ名,小カテゴリ名,商品コード,商品名,...,デフォルト原価率,原価率,在庫数,在庫数（不良品）,受注可能在庫数,引当待数,引当済数,入庫予定数,受注可能在庫金額,属性
0,2021/02/01,UNITED EAST TONAL倉庫,TONAL事業部,TN000,TONAL,トップス,ニット,ニット,A0TN00002UTM,アンゴラ混ペプラムＫＴ,...,100.00,< NULL >,0,0,4,0,0,< NULL >,52000.0,予約中在庫
1,2021/02/01,UNITED EAST TONAL倉庫,TONAL事業部,TN000,TONAL,ボトムス,パンツ,パンツ,A0TN0000888P,【JJ 3月号掲載】ストレッチツイルロングガウチョパンツ,...,100.00,< NULL >,0,0,2,0,0,< NULL >,30000.0,予約中在庫
2,2021/02/01,UNITED EAST TONAL倉庫,TONAL事業部,TN000,TONAL,トップス,ブラウス,ブラウス,A0TN00008SOG,ノースリーブブラウス×スリットタイトスカートSET,...,100.00,< NULL >,0,0,3,0,0,< NULL >,63000.0,予約中在庫
3,2021/02/01,UNITED EAST TONAL倉庫,TONAL事業部,TN000,TONAL,トップス,ブラウス,ブラウス,A0TN000091B3,ウエストマークブラウス,...,100.00,< NULL >,0,0,2,0,0,< NULL >,30000.0,予約中在庫
4,2021/02/01,UNITED EAST TONAL倉庫,TONAL事業部,TN000,TONAL,ボトムス,スカート,スカート,A0TN000091B4,三田 友梨佳 さん着用レースアーチヘムスカート,...,100.00,< NULL >,0,0,1,0,0,< NULL >,17000.0,予約中在庫
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25254,2021/02/01,< NULL >,109事業部,TL160,merryjenny,レディース,ワンピース,ワンピースその他,A0TL1000JC8R,【周年商品】8thワンピース,...,100,< NULL >,0,0,3,0,< NULL >,< NULL >,42000.0,予約中在庫
25255,2021/02/01,SGL,109事業部,TL177,109商品開発部★らすく,レディース,ファッション小物,小物その他,A0TL1000JMX9,【らすく】タンブラー,...,100,70.00,0,0,21,1,0,< NULL >,35700.0,予約中在庫
25256,2021/02/01,SGL,109事業部,TL177,109商品開発部★らすく,レディース,ファッション小物,小物その他,A0TL1000JMXA,【らすく】布ステッカー,...,100,70.00,0,0,13,0,0,< NULL >,6500.0,予約中在庫
25257,2021/02/01,SGL,109事業部,TL178,109商品開発部★pino,レディース,ファッション小物,カードケース,A0TL1000JT5T,【pino】パスケース,...,100,70.00,0,0,7,0,0,< NULL >,10500.0,予約中在庫


In [20]:
df_reservation = stockReport.groupby(["属性","事業部名" , "ブランドコード"]).agg({'受注可能在庫金額':'sum'})

In [21]:
df_reservation

受注可能在庫金額
属性    事業部名                 ブランドコード              
予約中在庫 109事業部               AB000    9.416050e+07
                           AB120    8.679060e+07
                           AL000    1.128000e+04
                           AL040    1.353685e+07
                           AL120    3.680400e+06
                           DR010    1.016000e+07
                           EG010    4.458814e+07
                           ENVYM    2.379700e+06
                           FV010    1.684980e+07
                           PC040    3.167900e+06
                           RW020    9.704580e+06
                           RW040    1.928510e+07
                           TL006    2.622099e+08
                           TL010    3.788281e+08
                           TL018    2.044081e+08
                           TL028    2.590326e+07
                           TL029    1.811009e+08
                           TL037    1.201334e+08
                           TL039    3.244782e+07
                           TL040    1.249480e+06
                           TL045    5.157609e+07
                           TL067    5.948320e+07
                           TL143    1.800000e+04
                           TL147    3.640761e+07
                           TL148    1.803650e+06
                           TL149    3.288431e+07
                           TL150    5.037790e+06
                           TL158    3.624336e+08
                           TL159    3.717230e+07
                           TL160    5.447130e+07
                           TL177    4.220000e+04
                           TL178    1.950000e+04
      AURALEE事業部           AU010    1.266323e+10
      Ailand事業部            AL010    1.330000e+05
                           AL020    8.540000e+05
                           ALNOV    0.000000e+00
      Arpege story事業部      DC000    2.528100e+06
                           DC010    1.646000e+06
                           DC020    4.478000e+05
                           DC050    9.740000e+05
      Brown&Street事業部      BR010    7.160000e+04
      CUBE SUGAR事業部        QS010    7.660000e+05
                           QS040    1.092000e+05
      DURAS事業部             DR010    5.280000e+05
      ECLIN事業部             ECLNV    0.000000e+00
      ELENDEEK事業部          MK510    2.465670e+07
                           MKNOV    0.000000e+00
      ETRE TOKYO事業部        ETNOV    0.000000e+00
      RUNWAY channel事業部    MK040    1.080000e+04
      Roomy's WEB STORE事業部 RW040    5.874000e+05
                           RW080    3.825000e+05
      SANKO BAZAAR EC事業部   SK010    7.383350e+05
                           SK090    8.900000e+03
      TONAL事業部             TN000    2.503300e+06
      UN3D.事業部             MK520    7.578000e+06
      atomicboxx事業部        AB120    9.200000e+03
      eimy istoire事業部      MI000    1.762380e+07
                           MINOV    0.000000e+00
      lifes-203事業部         ZL010    1.021520e+08
      willfully事業部         TS010    1.755600e+06
      西川産業事業部              NS001    1.500000e+02

In [22]:
df_reservation_con =pd.concat([df_reservation, ap_allStock, ap_stock_in3W, ap_stock_in3W_receive, ap_stock_nothing_agg], axis = 0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [23]:
df_reservation_con

SKU数      受注可能在庫金額
属性           事業部名                 ブランドコード                     
予約中在庫        109事業部               AB000      NaN  9.416050e+07
                                  AB120      NaN  8.679060e+07
                                  AL000      NaN  1.128000e+04
                                  AL040      NaN  1.353685e+07
                                  AL120      NaN  3.680400e+06
                                  DR010      NaN  1.016000e+07
                                  EG010      NaN  4.458814e+07
                                  ENVYM      NaN  2.379700e+06
                                  FV010      NaN  1.684980e+07
                                  PC040      NaN  3.167900e+06
                                  RW020      NaN  9.704580e+06
                                  RW040      NaN  1.928510e+07
                                  TL006      NaN  2.622099e+08
                                  TL010      NaN  3.788281e+08
                                  TL018      NaN  2.044081e+08
                                  TL028      NaN  2.590326e+07
                                  TL029      NaN  1.811009e+08
                                  TL037      NaN  1.201334e+08
                                  TL039      NaN  3.244782e+07
                                  TL040      NaN  1.249480e+06
                                  TL045      NaN  5.157609e+07
                                  TL067      NaN  5.948320e+07
                                  TL143      NaN  1.800000e+04
                                  TL147      NaN  3.640761e+07
                                  TL148      NaN  1.803650e+06
                                  TL149      NaN  3.288431e+07
                                  TL150      NaN  5.037790e+06
                                  TL158      NaN  3.624336e+08
                                  TL159      NaN  3.717230e+07
                                  TL160      NaN  5.447130e+07
                                  TL177      NaN  4.220000e+04
                                  TL178      NaN  1.950000e+04
             AURALEE事業部           AU010      NaN  1.266323e+10
             Ailand事業部            AL010      NaN  1.330000e+05
                                  AL020      NaN  8.540000e+05
                                  ALNOV      NaN  0.000000e+00
             Arpege story事業部      DC000      NaN  2.528100e+06
                                  DC010      NaN  1.646000e+06
                                  DC020      NaN  4.478000e+05
                                  DC050      NaN  9.740000e+05
             Brown&Street事業部      BR010      NaN  7.160000e+04
             CUBE SUGAR事業部        QS010      NaN  7.660000e+05
                                  QS040      NaN  1.092000e+05
             DURAS事業部             DR010      NaN  5.280000e+05
             ECLIN事業部             ECLNV      NaN  0.000000e+00
             ELENDEEK事業部          MK510      NaN  2.465670e+07
                                  MKNOV      NaN  0.000000e+00
             ETRE TOKYO事業部        ETNOV      NaN  0.000000e+00
             RUNWAY channel事業部    MK040      NaN  1.080000e+04
             Roomy's WEB STORE事業部 RW040      NaN  5.874000e+05
                                  RW080      NaN  3.825000e+05
             SANKO BAZAAR EC事業部   SK010      NaN  7.383350e+05
                                  SK090      NaN  8.900000e+03
             TONAL事業部             TN000      NaN  2.503300e+06
             UN3D.事業部             MK520      NaN  7.578000e+06
             atomicboxx事業部        AB120      NaN  9.200000e+03
             eimy istoire事業部      MI000      NaN  1.762380e+07
                                  MINOV      NaN  0.000000e+00
             lifes-203事業部         ZL010      NaN  1.021520e+08
             willfully事業部         TS010      NaN  1.755600e+06
             西川産業事業部              NS001      NaN  1.500000e+02
Arpege総在庫    Arpege story事業部      DC000      NaN  1.117155e+08
            

In [24]:
df_reservation_con.to_csv('予約中在庫.csv', encoding='utf_8_sig')